## 🎯 Desenvolvimento do Agente DQN para Alocação de Ativos

Este notebook implementa a Etapa 6 do projeto de Reinforcement Learning, dedicada à construção prática de um agente baseado no algoritmo Deep Q-Network (DQN), aplicado à alocação dinâmica de uma carteira com três ativos: VALE3, PETR4 e BRFS3.

O agente será treinado em um ambiente simulado (`PortfolioEnv`), configurado com base nos dados históricos processados previamente. A arquitetura será baseada em PyTorch com suporte a CUDA, validando os aprendizados teóricos sobre a função Q, replay buffer e políticas $\epsilon$-greedy.

Etapas implementadas neste notebook:

1. Verificação do ambiente e suporte à GPU;
2. Definição da rede neural DQN;
3. Construção do ambiente simulado `PortfolioEnv`;
4. Configuração do buffer de experiência e parâmetros de treinamento;
5. Execução do treinamento inicial do agente;
6. Análise dos resultados preliminares.


In [1]:
# Bibliotecas principais para RL e Data Science
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
from collections import deque
import matplotlib.pyplot as plt
import seaborn as sns

# Verifica se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Dispositivo ativo: {device}")
print(f"CUDA disponível? {torch.cuda.is_available()}")


✅ Dispositivo ativo: cuda
CUDA disponível? True


## 🧠 Arquitetura da Rede Neural DQN

A rede neural utilizada neste projeto segue uma arquitetura simples e eficiente, adequada ao tamanho do vetor de estado calculado com base em indicadores técnicos e variáveis da carteira.

A rede recebe como entrada o vetor de estado do ambiente simulado (`PortfolioEnv`) e retorna os valores Q estimados para cada ação possível. A estrutura adotada é composta por:

- Camada de entrada: dimensão igual ao tamanho total do vetor de estado;
- Camada oculta 1: 128 neurônios com ativação ReLU;
- Camada oculta 2: 64 neurônios com ativação ReLU;
- Camada de saída: 9 neurônios, correspondentes às ações discretas possíveis (comprar, vender ou manter) para cada um dos três ativos.

A saída da rede representa o valor esperado de recompensa (Q-value) para cada ação, dada a configuração atual do ambiente.


In [2]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)


## 🌍 Ambiente Simulado: `PortfolioEnv`

O ambiente `PortfolioEnv` simula o comportamento de um mercado financeiro com três ativos (VALE3, PETR4, BRFS3), considerando variáveis de estado, posições mantidas pelo agente e saldo de caixa disponível.

A interface segue o padrão `gym.Env`, com os principais métodos:

- `reset()`: reinicializa o ambiente para o primeiro dia útil, zera posições e caixa, e retorna o primeiro estado observável;
- `step(action_vector)`: executa as ações sobre os ativos, atualiza posições, saldo e calcula a recompensa;
- `render()`: imprime informações do estado atual da carteira — útil para fins de depuração e visualização em tempo real durante o treinamento;
- Atributos internos controlam a linha temporal, recompensas acumuladas e histórico da carteira.

As ações são vetores discretos com tamanho igual ao número de ativos. Para cada ativo, a ação pode ser:

- `0`: manter posição;
- `1`: comprar uma unidade (caso haja saldo);
- `2`: vender uma unidade (caso haja posição).

A recompensa é definida com base na variação do valor total da carteira entre `t` e `t+1`, incluindo caixa e valor de mercado das posições. Custos de transação podem ser incorporados como penalidade.


In [6]:
import gym
from gym import spaces
import numpy as np

class PortfolioEnv(gym.Env):
    def __init__(self, df, initial_cash=1.0):
        super(PortfolioEnv, self).__init__()

        # DataFrame com os dados de mercado históricos (já preparados externamente)
        self.df = df.reset_index(drop=True)

        self.n_assets = 3  # Número de ativos: VALE3, PETR4, BRFS3
        self.initial_cash = initial_cash  # Valor inicial em caixa

        # Espaço de ações: vetor com 3 entradas (uma por ativo), onde:
        # 0 = manter, 1 = comprar, 2 = vender
        self.action_space = spaces.MultiDiscrete([3] * self.n_assets)

        # Inicializações
        self.current_step = 0           # Índice temporal do ambiente
        self.cash = self.initial_cash   # Saldo disponível
        self.positions = [0] * self.n_assets  # Quantidade de cada ativo na carteira
        self.history = []               # Histórico de recompensas ou estados, se desejado

    def reset(self):
        """Reinicia o ambiente para o primeiro dia útil da simulação"""
        self.current_step = 0
        self.cash = self.initial_cash
        self.positions = [0] * self.n_assets
        self.history.clear()
        return self._get_state()

    def step(self, action):
        """
        Aplica ações sobre os ativos e avança para o próximo dia.
        A lógica detalhada será implementada posteriormente.
        """
        next_state = self._get_state()
        reward = 0.0
        done = self.current_step >= len(self.df) - 2  # -2 para garantir que t+1 exista
        info = {}
        self.current_step += 1
        return next_state, reward, done, info

    def _get_state(self):
        """Retorna o vetor de estado do dia atual, combinando dados de mercado com portfólio"""
        market_data = self.df.iloc[self.current_step].values
        return np.concatenate([market_data, self.positions, [self.cash]])

    def render(self, mode='human'):
        """
        Exibe o estado atual do ambiente (modo 'human' é um padrão da API gym).
        Pode ser ignorado se rodando em scripts automatizados.
        """
        print(f"Step: {self.current_step} | Caixa: {self.cash:.2f} | Posições: {self.positions}")


## 🔁 Lógica de Transição do Ambiente: Implementação do Método `step()`

O método `step()` é o coração do ambiente `PortfolioEnv`. Ele define como o agente interage com o mercado e como essa interação afeta o estado da carteira. A implementação segue estas etapas:

1. **Registro do valor da carteira antes das ações**  
   Calcula o valor da carteira no tempo `t`, somando o caixa disponível e o valor de mercado das posições atuais com os preços do dia corrente.

2. **Execução das ações de compra, venda ou manutenção**  
   Para cada ativo:
   - `1` (compra): se houver saldo suficiente, reduz o caixa e incrementa a posição;
   - `2` (venda): se houver posição, reduz a quantidade do ativo e adiciona o valor ao caixa;
   - `0` (manter): nenhuma alteração.

   Cada operação de compra ou venda acarreta um **custo de transação fixo**, deduzido do caixa disponível. Isso reflete taxas de corretagem, spread e custos operacionais — incentivando o agente a evitar operações excessivas.

3. **Cálculo da recompensa**  
   Após as ações, avança para o próximo dia (`t+1`) e calcula o novo valor da carteira.  
   A recompensa é definida como a **variação percentual do valor da carteira** entre `t` e `t+1`.  
   Se o episódio estiver no último dia útil (`t+1` não existir), a recompensa é zero.

4. **Atualização do passo temporal e retorno dos resultados**  
   - `next_state`: novo vetor de observação;
   - `reward`: feedback para o agente;
   - `done`: sinaliza fim do episódio;
   - `info`: dicionário com dados complementares como o valor atual da carteira.

Este método transforma a simulação em um ambiente dinâmico realista, no qual o agente precisa aprender a alocar recursos de forma estratégica e econômica, ponderando retorno e custo de transação.


In [7]:
def step(self, action):
    """
    Aplica o vetor de ações fornecido (um por ativo), atualiza as posições e o caixa,
    avança o tempo e calcula a recompensa como a variação percentual da carteira líquida.
    """

    # Definição do custo fixo por transação (compra ou venda)
    transaction_cost = 0.001  # 0.1% do valor da operação

    # Preços atuais dos ativos (tempo t)
    current_prices = self.df.iloc[self.current_step].values

    # Valor da carteira antes das ações
    portfolio_value_before = self.cash + sum([
        self.positions[i] * current_prices[i]
        for i in range(self.n_assets)
    ])

    # Aplicação das ações
    for i in range(self.n_assets):
        price = current_prices[i]
        if action[i] == 1:  # Comprar
            total_cost = price * (1 + transaction_cost)
            if self.cash >= total_cost:
                self.positions[i] += 1
                self.cash -= total_cost


## ✅ Finalização do Ambiente Simulado `PortfolioEnv`

Com a implementação completa do método `step()`, o ambiente `PortfolioEnv` está agora totalmente funcional e pronto para ser utilizado no treinamento do agente de Reinforcement Learning.

Esse ambiente representa, com realismo e controle, um cenário simplificado de mercado no qual o agente:

- Observa indicadores técnicos e seu próprio portfólio (estado);
- Decide diariamente se deve comprar, vender ou manter posição (ação);
- Recebe uma recompensa baseada na evolução da carteira (retorno financeiro ajustado por custo de transação).

A inclusão de **custos fixos por operação** torna o desafio mais alinhado com o mundo real, penalizando estratégias com excesso de transações e incentivando decisões eficientes.

Além disso, o ambiente já é compatível com algoritmos baseados em `gym`, podendo ser diretamente integrado ao loop de treinamento do DQN, Replay Buffer e política de exploração ε-greedy.

A partir deste ponto, seguimos para a configuração da estrutura de aprendizado propriamente dita, que inclui:

1. Buffer de experiências (Replay Buffer);
2. Política de exploração baseada em ε-greedy;
3. Loop de treinamento com atualização da rede Q e rede alvo.

Com isso, encerramos a Etapa 6.3 — *Definição do Ambiente de Simulação*.
